In [1]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai langchainhub duckduckgo-search wikipedia

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# **ReAct Prompting Framework Overview**

The ReAct framework was [introduced in a 2022 paper, ReAct: Synergizing Reasoning and Acting in Language Models, by researchers at Google](https://blog.research.google/2022/11/react-synergizing-reasoning-and-acting.html).

### 🌐 **ReAct (Reasoning + Acting) Explained**

- Blends reasoning with action in language models.

- Generates reasoning traces alongside task-specific actions.

### 🔗 **Capabilities of ReAct**

1. 🔄 **Interleaved Reasoning and Acting**: Fuses reasoning with actions for fluid updates.

2. 🌍 **External Interactions**: Accesses external sources like Wikipedia for more info.

3. 🧑‍🎨 **Human-like Problem Solving**: Offers easy-to-follow, logical trajectories.

4. 🛠️ **Wide-Ranging Use Cases**: Ideal for varied tasks like QA, text games, web browsing.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03629/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>Comparison of four prompting methods, (a) Standard, (b) Chain-of-thought (CoT, Reason Only), (c) Act-only, and (d) (Reason+Act), solving a HotpotQA question. (2) Comparison of (a) Act-only and (b) prompting methods to solve an interactive AlfWorld task. In both domains, the initial prompts of in-context examples are ommitted, and only show task solving trajectories generated by the model (Act, Thought) and the environment (Obs).</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03629">Image Source</a>
</figure>


### 🌟 **Why ReAct Stands Out**

1. **Balanced Emphasis on Reasoning & Acting**: Surpasses models limited to just reasoning or action.

2. **Engages with External Data**: Pulls in real-time info for better accuracy.

3. **Boosted Transparency**: Step-by-step approach clarifies decision-making.


# 🤖 **Overview of ReAct in LangChain**

- ReAct is an agent type in LangChain.

- It uses the ReAct framework to pick tools based on their descriptions.

- Useful for chatbots, document question answering, etc.

- Context-aware for better response generation.

## 🚀 **Steps to Implement an Agent**

1. ✍️ **Prompt Template Creation**: Guide the agent on tasks and output formatting.

2. 🧩 **Output Parser Definition**: Convert LLM output into actionable responses.

3. 🧠 **LLM Setup**: Choose the preferred language model.

4. 🛑 **Stop Sequence Definition**: Instruct the LLM when to stop output.

5. 🤹 **Agent Setup**: Merge LLM, prompt template, output parser, and stop sequence.

6. 🏃 **Agent Usage**: Operate the agent using an executor with user inputs.

In [3]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchResults
from langchain import Wikipedia

In [4]:
search = DuckDuckGoSearchResults()

search_tool = Tool(
    name="Current Event Search",
    description = "Useful for when you need to search about current events.",
    func=search.run
)

wiki = Wikipedia()

wiki_tool = Tool(
    name="Noun Search",
    description = "Useful for when you need to search for information about nouns.",
    func=wiki.search,
)

tool_belt = [search_tool, wiki_tool]

In [5]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [6]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [7]:
prompt.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [8]:
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0.0)

In [9]:
agent = create_react_agent(llm, tool_belt, prompt)

In [10]:
agent_executor = AgentExecutor(agent=agent,
                               tools=tool_belt,
                               verbose=True,
                               handle_parsing_errors=True)

In [11]:
agent_executor.invoke({"input":"Who is playing in the Super Bowl in 2024? Where will it be played?"})



> Entering new AgentExecutor chain...
I need to find out which teams are playing in the Super Bowl in 2024 and the location of the game.
Action: Current Event Search
Action Input: Super Bowl 2024 teams and location

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


[snippet: Redirecting to /nfl/news/where-is-2024-super-bowl-location-stadium-date-time-tv-streaming-how-to-watch-49ers-vs-chiefs/ ., title: Where is 2024 Super Bowl: Location, stadium, date, time, TV, streaming ..., link: https://www.cbssports.com/nfl/news/where-is-2024-super-bowl-date-location-stadium-tv-streaming-how-to-watch/], [snippet: This year, the Super Bowl will be played Feb. 11 at Allegiant Stadium in Las Vegas. The state-of-the-art venue in Sin City is home to the Las Vegas Raiders, who will not be playing for the big..., title: Who is in the Super Bowl 2024? Breaking down NFL playoff bracket, link: https://www.usatoday.com/story/sports/nfl/2024/01/16/who-is-playing-in-super-bowl-nfl-championship/72242712007/], [snippet: The game is being held in Allegiant Stadium, which was billed as the second-most expensive stadium in the world at roughly $1.9 billion, $750 million of which was reportedly shouldered by taxpayers. As the 2023 NFL season comes to a close, the 2024 NFL Draf

{'input': 'Who is playing in the Super Bowl in 2024? Where will it be played?',
 'output': 'The Kansas City Chiefs and San Francisco 49ers are playing in the Super Bowl in 2024. It will be played at Allegiant Stadium in Las Vegas on February 11.'}